In [3]:
#!pip install pandas-datareader

In [2]:
import numpy as np
import pandas as pd
import pandas_datareader
import datetime as dt
import os

PATH = os.getcwd()

In [3]:
#Grab sentiments
#Big-cap
big_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USbig_Sent12_21.csv'))
big_sent_all['date'] = pd.to_datetime(big_sent_all['date']).dt.date
#Mid-cap
mid_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USmed_Sent12_21.csv'))
mid_sent_all['date'] = pd.to_datetime(mid_sent_all['date']).dt.date
#Small-caps
small_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USsmall_Sent12_21.csv'))
small_sent_all['date'] = pd.to_datetime(small_sent_all['date']).dt.date

## Sentiment Indicators

In [4]:
## The sentiment dataset in the 'dataSent12_21Good' folder is in long format, with each column being a different sentiment indicator.
# We create a table for each sentiment indicator, and pivot them to into familiar wide format:
# Each table will align all stocks by date and those which has no data for given date will be fill with NaN

#Make a table for each sentiment indicator
big_sent_tables = {}
big_sent_nan_tables = {}
for i in big_sent_all.columns[2:]:   
    big_sent_pivot = big_sent_all.pivot(index="date", columns="stock", values= i)
    big_sent_pivot.index = pd.to_datetime(big_sent_pivot.index)
    big_sent_tables['big_'+i] = big_sent_pivot
    big_sent_pivot.to_csv(os.path.join(PATH,'Tables','big_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder

print('List of tables created: ',big_sent_tables.keys())
print('RCV Table:')
big_sent_tables['big_RCV'].head()

List of tables created:  dict_keys(['big_RCV', 'big_RVT', 'big_positivePartscr', 'big_negativePartscr', 'big_splogscr', 'big_linscr'])
RCV Table:


stock,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
2012-01-03,NaN,0.000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,...,2.632,NaN,NaN,0.000,NaN,0.000,0.000,0.000,NaN,33.333
2012-01-04,0.000,41.667,NaN,44.444,-14.286,0.000,0.000,0.000,0.000,0.000,...,47.692,0.000,0.000,30.000,0.000,22.222,0.000,37.500,0.0,43.478
2012-01-05,25.000,45.455,NaN,0.000,33.333,38.889,13.333,-7.692,4.167,-9.091,...,38.571,0.000,25.000,57.333,38.462,49.383,51.852,48.148,-20.0,38.889
2012-01-06,46.666,45.395,NaN,-33.333,-73.333,57.384,-60.000,-43.750,41.935,-35.714,...,26.250,33.335,-57.142,27.941,-28.571,14.706,36.765,54.412,NaN,20.000


In [5]:
#Same procedure for mid and small-cap

#Mid companies
mid_sent_tables = {}
mid_sent_nan_tables = {}
for i in mid_sent_all.columns[2:]:   
    mid_sent_pivot = mid_sent_all.pivot(index="date", columns="stock", values= i)
    mid_sent_pivot.index = pd.to_datetime(mid_sent_pivot.index)
    mid_sent_tables['mid_'+i] = mid_sent_pivot
    mid_sent_pivot.to_csv(os.path.join(PATH,'Tables','mid_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder
     
#Small companies
small_sent_tables = {}
small_sent_nan_tables = {}
for i in small_sent_all.columns[2:]:   
    small_sent_pivot = small_sent_all.pivot(index="date", columns="stock", values= i)
    small_sent_pivot.index = pd.to_datetime(small_sent_pivot.index)
    small_sent_tables['small_'+i] = small_sent_pivot
    small_sent_pivot.to_csv(os.path.join(PATH,'Tables','small_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder

## Stock Price Data

In [6]:
big_tickers = big_sent_tables['big_RCV'].columns
mid_tickers = mid_sent_tables['mid_RCV'].columns
small_tickers = small_sent_tables['small_RCV'].columns

In [8]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
big_tickers = big_sent_tables['big_RCV'].columns.drop('ABBV') # ABBV returns weekly returns starting on Tuesday in  for some unknown reason

# We would like all available data from 2012/01/02 until 2021/12/01.
start_date = big_sent_tables['big_RCV'].index[0].strftime('%Y-%m-%d') 
end_date = big_sent_tables['big_RCV'].index[-1].strftime('%Y-%m-%d')

# User pandas_reader to load the desired data in a weekly format. As simple as that.
big_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(big_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
big_panel_data = pd.DataFrame(big_panel_data)
#big_panel_data.to_csv(os.path.join(PATH,'Tables','big_prices.csv')) #Store in csv format in the 'Tables' folder
big_panel_data.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,5.279352,12.972254,32.785950,21.759474,38.519989,19.506790,5.43,182.610001,41.661335,5.321596,...,18.209999,44.973885,54.356098,21.127947,23.387878,24.163364,21.554792,46.596504,12.920537,55.891048
2012-01-09,5.628166,12.892712,32.114777,21.591969,38.569134,20.675205,5.66,178.419998,43.107704,5.691868,...,18.430000,44.905727,54.859089,22.102499,23.394844,24.854429,22.053812,47.022987,13.735711,55.733452
2012-01-16,6.005263,12.907758,34.060341,21.907923,39.860619,21.255270,6.42,190.929993,43.350266,6.087974,...,19.330000,44.539322,55.791195,21.881701,23.357691,24.886356,22.746492,48.183941,14.027818,57.447227
2012-01-23,7.711625,13.736339,33.117283,21.617182,38.842865,20.923809,6.82,195.369995,43.185661,6.277415,...,23.090000,43.474190,56.249825,21.211702,23.466835,23.762419,22.046370,47.947014,13.905539,56.357254
2012-01-30,9.088029,14.117155,33.329678,21.609324,38.962196,22.514845,7.08,187.679993,45.264812,6.751020,...,24.969999,43.721306,56.738041,22.231934,24.855572,24.164743,22.813524,48.989510,14.469367,55.759727


In [9]:
#Same procedure for mid and small
#Mid-caps
mid_tickers = mid_sent_tables['mid_RCV'].columns
start_date = mid_sent_tables['mid_RCV'].index[0].strftime('%Y-%m-%d')
end_date = mid_sent_tables['mid_RCV'].index[-1].strftime('%Y-%m-%d')
mid_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(mid_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
mid_panel_data = pd.DataFrame(mid_panel_data)
#mid_panel_data.to_csv(os.path.join(PATH,'Tables','mid_prices.csv')) #Store in csv format in the 'Tables' folder

/opt/homebrew/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'ADS', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/homebrew/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'PCI', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/opt/homebrew/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'TCO', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [10]:
#Small-caps
small_tickers = small_sent_tables['small_RCV'].columns
start_date = small_sent_tables['small_RCV'].index[0].strftime('%Y-%m-%d')
end_date = small_sent_tables['small_RCV'].index[-1].strftime('%Y-%m-%d')
small_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(small_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
small_panel_data = pd.DataFrame(small_panel_data)
#small_panel_data.to_csv(os.path.join(PATH,'Tables','small_prices.csv')) #Store in csv format in the 'Tables' folder

/opt/homebrew/lib/python3.9/site-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'INTL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [156]:
big_panel_data.describe()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,...,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,29.853003,46.208520,76.403275,55.778054,100.567580,43.311936,23.769633,1228.443396,86.017446,20.245517,...,54.004903,172.671980,98.232929,38.866198,106.222788,40.722022,39.716230,82.847009,24.377418,59.714999
std,13.165709,37.902158,22.562542,29.474035,48.648670,9.642800,31.566222,1038.071789,30.405967,9.447480,...,20.889458,108.886416,35.902708,9.485844,63.517024,8.993890,8.890773,29.136350,4.951926,8.584846
min,5.279352,12.119164,30.683222,21.506247,35.505150,18.120352,1.670000,178.419998,41.543427,5.321594,...,17.780001,43.536598,52.966915,20.957052,23.405643,23.475946,20.970749,46.494160,12.920536,28.654728
25%,17.079329,21.081754,64.160900,33.962798,59.090282,38.240262,3.650000,332.880005,65.314680,13.154113,...,37.352500,72.228413,78.439589,32.618833,51.127288,34.041476,32.530491,61.721056,21.400329,57.597895
50%,31.701316,29.332281,79.777470,41.319750,87.409885,45.946999,9.840000,809.265015,78.896275,19.514085,...,53.095001,147.995499,90.945660,36.268644,79.447773,38.961329,42.324793,68.720882,24.132570,61.392267
75%,40.459659,51.459453,88.960617,75.740543,138.991318,50.422192,29.515001,1816.377502,100.601908,26.946706,...,69.429998,246.156433,104.375635,46.205420,159.927593,50.045856,46.344869,102.208635,27.132939,65.256172
max,56.988731,164.560349,126.690041,127.927086,235.679306,59.332623,155.410004,3719.340088,186.083435,47.319759,...,96.699997,457.618835,212.720993,61.726746,247.840775,57.480377,58.148563,150.261490,37.384274,72.813080


## Log Retruns

In [92]:
#Log return:
big_log_ret = np.log(big_panel_data) - np.log(big_panel_data.shift(1))
big_log_ret.index = big_log_ret.index.shift(-7, freq='D')
big_log_ret.drop(index= big_log_ret.index[0], axis = 0, inplace=True)
#big_log_ret.to_csv(os.path.join(PATH,'Tables','big_log_ret.csv')) #Store in csv format in the 'Tables' folder

#Percentage change:
#big_pct_change = big_panel_data.pct_change()
#big_pct_changet.to_csv(os.path.join(PATH,'Tables','big_pct_change.csv'))

big_log_ret.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,0.063980,-0.006151,-0.020684,-0.007728,0.001275,0.058173,0.041485,-0.023212,0.034128,0.067265,...,0.012009,-0.001517,0.009211,0.045094,0.000298,0.028198,0.022887,0.009111,0.061181,-0.002824
2012-01-09,0.064853,0.001166,0.058817,0.014527,0.032937,0.027670,0.125994,0.067767,0.005611,0.067277,...,0.047679,-0.008193,0.016848,-0.010040,-0.001589,0.001284,0.030925,0.024389,0.021043,0.030286
2012-01-16,0.250093,0.062216,-0.028078,-0.013360,-0.025864,-0.015717,0.060441,0.022988,-0.003804,0.030643,...,0.177741,-0.024205,0.008187,-0.031098,0.004662,-0.046214,-0.031263,-0.004929,-0.008755,-0.019156
2012-01-23,0.164229,0.027346,0.006393,-0.000364,0.003067,0.073287,0.037414,-0.040157,0.047021,0.072735,...,0.078275,0.005668,0.008642,0.046977,0.057494,0.016789,0.034206,0.021510,0.039747,-0.010659
2012-01-30,-0.085504,0.070830,-0.016189,0.001998,-0.027394,-0.018949,-0.004246,-0.011468,-0.008457,0.028915,...,-0.050093,0.038426,-0.000130,-0.006528,0.062212,-0.003972,-0.008037,-0.002098,-0.061975,-0.013277


In [16]:
big_log_ret.index

DatetimeIndex(['2012-01-02', '2012-01-09', '2012-01-16', '2012-01-23',
               '2012-01-30', '2012-02-06', '2012-02-13', '2012-02-20',
               '2012-02-27', '2012-03-05',
               ...
               '2021-09-27', '2021-10-04', '2021-10-11', '2021-10-18',
               '2021-10-25', '2021-11-01', '2021-11-08', '2021-11-15',
               '2021-11-22', '2021-11-29'],
              dtype='datetime64[ns]', name='Date', length=518, freq=None)

In [121]:
def big_mean_sentiment(sentiment:str, tolerance = 2):
    '''
    Function to aggregate sentiment data by week (indexed by big_log_retruns index) calculates the mean of a week's worth of sentiment.
    Can be modified to accept more or less NaN's in a week, default is 2.

    Returns a matrix with the same index as big_log_ret which corresponds to our observed ranking
    '''

    big_sentiment = big_sent_tables[sentiment]
    columns = big_sentiment.columns
    rows = []
    for i in range(len(big_log_ret.index)-1):
        step1 = big_sentiment[big_log_ret.index[i]:big_log_ret.index[i+1]-dt.timedelta(days=1)]
        trial = np.zeros(columns.shape)
        for idx, column in enumerate(columns):
            if step1[column].isnull().sum() > tolerance:
                trial[idx] = np.NaN
            else:
                trial[idx] = step1[column].mean(skipna = True)
        rows.append(trial)
    trial = np.zeros(columns.shape)
    step2 = big_sentiment[big_log_ret.index[-1]:big_log_ret.index[-1] + dt.timedelta(days = 6)]
    for idx, column in enumerate(columns):
        if step2[column].isnull().sum() > tolerance:
            trial[idx] = np.NaN
        else:
            trial[idx] = step2[column].mean(skipna = True)
    rows.append(trial)


    df = pd.DataFrame(rows, columns=big_sentiment.columns)
    df.index = big_log_ret.index
    return df

In [106]:
def big_max_sentiment(sentiment:str, tolerance = 2):
    '''
    Function to aggregate sentiment data by week (indexed by big_log_retruns index) calculates the maximum of a week's worth of sentiment.
    Can be modified to accept more or less NaN's in a week, default is 2.

    Returns a matrix with the same index as big_log_ret which corresponds to our observed ranking
    '''

    big_sentiment = big_sent_tables[sentiment]
    columns = big_sentiment.columns
    rows = []
    for i in range(len(big_log_ret.index) -1):
        step1 = big_sentiment[big_log_ret.index[i]:big_log_ret.index[i+1] - dt.timedelta(days=1)]
        trial = np.zeros(columns.shape)
        for idx, column in enumerate(columns):
            if step1[column].isnull().sum() > tolerance:
                trial[idx] = np.NaN
            else:
                trial[idx] = step1[column].max()
        rows.append(trial)
    trial = np.zeros(columns.shape)
    step2 = big_sentiment[big_log_ret.index[-1]:big_log_ret.index[-1] + dt.timedelta(days = 6)]
    for idx, column in enumerate(columns):
        if step2[column].isnull().sum() > tolerance:
            trial[idx] = np.NaN
        else:
            trial[idx] = step2[column].max()
    rows.append(trial)

    df = pd.DataFrame(rows, columns=big_sentiment.columns)
    df.index = big_log_ret.index
    return df


In [120]:
def big_min_sentiment(sentiment:str, tolerance = 2):
    '''
    Function to aggregate sentiment data by week (indexed by big_log_retruns index) calculates the minimum of a week's worth of sentiment.
    Can be modified to accept more or less NaN's in a week, default is 2.

    Returns a matrix with the same index as big_log_ret which corresponds to our observed ranking
    '''
    big_sentiment = big_sent_tables[sentiment]
    columns = big_sentiment.columns
    rows = []
    for i in range(len(big_log_ret.index) -1):
        step1 = big_sentiment[big_log_ret.index[i]:big_log_ret.index[i+1] - dt.timedelta(days=1)]
        trial = np.zeros(columns.shape)
        for idx, column in enumerate(columns):
            if step1[column].isnull().sum() > tolerance:
                trial[idx] = np.NaN
            else:
                trial[idx] = step1[column].min()
        rows.append(trial)
    trial = np.zeros(columns.shape)
    step2 = big_sentiment[big_log_ret.index[-1]:big_log_ret.index[-1] + dt.timedelta(days = 6)]
    for idx, column in enumerate(columns):
        if step2[column].isnull().sum() > tolerance:
            trial[idx] = np.NaN
        else:
            trial[idx] = step2[column].min()
    rows.append(trial)

    df = pd.DataFrame(rows, columns=big_sentiment.columns)
    df.index = big_log_ret.index[:-1]
    return df


In [122]:
big_linscr_mean = big_mean_sentiment("big_linscr")
big_linscr_mean.to_csv(os.path.join(PATH,'Tables','big_linscr_mean.csv'))

In [123]:
big_linscr_mean.tail()

stock,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-25,46.559083,32.474271,39.217117,54.785571,61.44512,38.76512,51.095533,58.297283,39.488743,32.917867,...,54.22980,66.06406,42.347780,61.799267,41.517433,51.196650,29.271743,57.352933,40.0000,51.421343
2021-11-01,29.841740,42.823500,37.000033,49.182617,50.94392,63.38036,59.381060,72.946200,53.312783,27.996833,...,42.85610,52.98864,52.196833,72.085040,47.600360,45.083740,33.516717,40.389883,40.0000,56.956660
2021-11-08,33.506867,39.823450,45.192140,41.996350,49.52610,71.18925,52.576680,71.762229,51.110600,19.656100,...,53.54514,33.20624,49.384567,66.564683,33.914140,50.169233,31.468771,70.813883,23.3008,42.531950
2021-11-15,23.310250,44.248800,42.072060,53.351643,49.41964,73.70314,48.695800,80.232886,46.799900,33.426600,...,12.12906,26.24382,53.390300,47.506600,23.941180,40.724950,30.282343,45.305786,10.0000,43.201717
2021-11-22,20.067150,45.430929,56.144100,48.903620,66.51820,27.71554,NaN,86.595986,46.718471,NaN,...,14.67994,NaN,46.677617,68.229333,NaN,28.958114,15.077883,50.108929,NaN,NaN


In [98]:
big_log_ret

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,0.063980,-0.006151,-0.020684,-0.007728,0.001275,0.058173,0.041485,-0.023212,0.034128,0.067265,...,0.012009,-0.001517,0.009211,0.045094,0.000298,0.028198,0.022887,0.009111,0.061181,-0.002824
2012-01-09,0.064853,0.001166,0.058817,0.014527,0.032937,0.027670,0.125994,0.067767,0.005611,0.067277,...,0.047679,-0.008193,0.016848,-0.010040,-0.001589,0.001284,0.030925,0.024389,0.021043,0.030286
2012-01-16,0.250093,0.062216,-0.028078,-0.013360,-0.025864,-0.015717,0.060441,0.022988,-0.003804,0.030643,...,0.177741,-0.024205,0.008187,-0.031098,0.004662,-0.046214,-0.031263,-0.004929,-0.008755,-0.019156
2012-01-23,0.164229,0.027346,0.006393,-0.000364,0.003067,0.073287,0.037414,-0.040157,0.047021,0.072735,...,0.078275,0.005668,0.008642,0.046977,0.057494,0.016789,0.034206,0.021510,0.039747,-0.010659
2012-01-30,-0.085504,0.070830,-0.016189,0.001998,-0.027394,-0.018949,-0.004246,-0.011468,-0.008457,0.028915,...,-0.050093,0.038426,-0.000130,-0.006528,0.062212,-0.003972,-0.008037,-0.002098,-0.061975,-0.013277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-25,0.127917,0.009831,0.044480,-0.035698,0.022508,-0.002202,0.125745,0.042540,0.016607,-0.017098,...,0.133125,-0.010172,-0.024901,0.005287,0.022875,-0.014255,-0.014173,0.005406,0.070522,0.008495
2021-11-01,-0.072699,-0.008564,-0.017635,0.032279,0.005776,-0.012802,0.081317,0.001749,0.029005,-0.001278,...,-0.056188,0.006952,0.023589,-0.003136,-0.021365,0.001912,0.014110,-0.016578,-0.037209,-0.018628
2021-11-08,-0.051060,0.068037,-0.035548,-0.012613,0.029778,-0.057633,0.049598,0.042057,-0.046994,-0.032719,...,-0.077587,-0.042254,-0.022389,-0.049294,-0.052644,-0.028684,-0.042901,-0.037020,0.029614,-0.037268


In [118]:
big_sent_tables["big_linscr"].tail(18)

stock,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
date,,,,,,,,,,,,,,,,,,,,,
2021-11-14,19.9319,38.5744,NaN,18.2670,NaN,100.0000,NaN,80.1111,53.1847,NaN,...,NaN,NaN,31.6574,50.0000,NaN,0.0000,0.0000,100.0000,NaN,12.4620
2021-11-15,NaN,NaN,NaN,64.6904,NaN,NaN,NaN,100.0000,60.3152,NaN,...,NaN,NaN,41.7249,0.0000,NaN,0.0000,0.0000,40.0000,NaN,NaN
2021-11-16,0.0000,40.7200,31.5201,73.9193,80.0659,69.7516,67.5895,84.6569,55.0357,21.2008,...,0.0000,25.9546,52.9359,66.1319,0.0000,23.4082,37.1295,10.9854,0.0000,52.9107
2021-11-17,30.6147,57.5039,55.3639,54.9790,56.5075,90.2235,55.2395,77.6961,39.5957,66.7014,...,50.0000,85.5356,67.5974,84.5390,90.2448,85.4158,50.7388,49.8122,50.0000,62.4465
2021-11-18,23.5548,46.3775,43.5464,36.6368,48.7776,58.0087,42.2468,80.9779,49.3891,35.0988,...,5.0597,0.0000,26.3381,57.2677,20.2114,66.1548,44.3975,60.2423,0.0000,55.6241
2021-11-19,56.4943,45.2053,54.9753,43.8700,22.4398,100.0000,40.6897,66.0155,59.9037,28.1208,...,0.0000,0.0000,35.4822,74.3104,9.2497,69.3709,26.9517,73.8821,0.0000,24.4926
2021-11-20,19.0170,34.9969,24.9546,51.2780,39.3074,50.5319,37.7135,75.5305,31.7087,16.0112,...,5.5856,19.7289,49.6536,50.2972,0.0000,0.0000,52.7589,37.6841,0.0000,13.7364
2021-11-21,10.1807,40.6892,NaN,48.0880,NaN,NaN,NaN,76.7533,31.6512,NaN,...,NaN,NaN,100.0000,0.0000,NaN,NaN,0.0000,44.5344,NaN,50.0000
2021-11-22,NaN,36.4162,NaN,NaN,NaN,NaN,NaN,100.0000,33.5544,NaN,...,NaN,NaN,NaN,57.0605,NaN,33.3333,0.0000,33.3333,NaN,NaN


In [88]:
big_linscr_mean.tail()
#big_log_ret.tail()

stock,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-25,46.559083,32.474271,39.217117,54.785571,61.44512,38.76512,51.095533,58.297283,39.488743,32.917867,...,54.22980,66.06406,42.347780,61.799267,41.517433,51.196650,29.271743,57.352933,40.0000,51.421343
2021-11-01,29.841740,42.823500,37.000033,49.182617,50.94392,63.38036,59.381060,72.946200,53.312783,27.996833,...,42.85610,52.98864,52.196833,72.085040,47.600360,45.083740,33.516717,40.389883,40.0000,56.956660
2021-11-08,33.506867,39.823450,45.192140,41.996350,49.52610,71.18925,52.576680,71.762229,51.110600,19.656100,...,53.54514,33.20624,49.384567,66.564683,33.914140,50.169233,31.468771,70.813883,23.3008,42.531950
2021-11-15,23.310250,44.248800,42.072060,53.351643,49.41964,73.70314,48.695800,80.232886,46.799900,33.426600,...,12.12906,26.24382,53.390300,47.506600,23.941180,40.724950,30.282343,45.305786,10.0000,43.201717
2021-11-22,20.067150,45.430929,56.144100,48.903620,66.51820,27.71554,NaN,86.595986,46.718471,NaN,...,14.67994,NaN,46.677617,68.229333,NaN,28.958114,15.077883,50.108929,NaN,NaN


In [90]:
#Same for mid and small-caps
#Log return:
mid_log_ret = np.log(mid_panel_data) - np.log(mid_panel_data.shift(1))
#mid_log_ret.to_csv(os.path.join(PATH,'Tables','mid_log_ret.csv')) #Store in csv format in the 'Tables' folder
small_log_ret = np.log(small_panel_data) - np.log(small_panel_data.shift(1))
#small_log_ret.to_csv(os.path.join(PATH,'Tables','small_log_ret.csv')) #Store in csv format in the 'Tables' folder

#Percentage change:
#mid_pct_change = mid_panel_data.pct_change()
#mid_pct_changet.to_csv(os.path.join(PATH,'Tables','mid_pct_change.csv'))
#small_pct_change = small_panel_data.pct_change()
#small_pct_changet.to_csv(os.path.join(PATH,'Tables','small_pct_change.csv'))

## Ranking Tables

In [165]:
big_rankings = big_log_ret.rank(1, ascending=False, method='first')
#big_rankings.to_csv(os.path.join(PATH,'Tables','big_rankings.csv')) #Store in csv format in the 'Tables' folder

#Same for Mid and Small-caps
mid_rankings = mid_log_ret.rank(1, ascending=False, method='first')
#mid_rankings.to_csv(os.path.join(PATH,'Tables','mid_rankings.csv')) #Store in csv format in the 'Tables' folder
small_rankings = small_log_ret.rank(1, ascending=False, method='first')
#small_rankings.to_csv(os.path.join(PATH,'Tables','small_rankings.csv')) #Store in csv format in the 'Tables' folder
big_rankings.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,10.0,88.0,95.0,89.0,78.0,13.0,28.0,98.0,33.0,9.0,...,63.0,82.0,65.0,21.0,79.0,36.0,45.0,66.0,12.0,86.0
2012-01-16,11.0,76.0,15.0,60.0,31.0,41.0,2.0,8.0,71.0,9.0,...,21.0,90.0,55.0,93.0,82.0,75.0,34.0,46.0,53.0,38.0
2012-01-23,1.0,9.0,92.0,73.0,90.0,75.0,10.0,25.0,55.0,16.0,...,3.0,88.0,42.0,95.0,45.0,102.0,96.0,57.0,66.0,79.0
2012-01-30,1.0,53.0,82.0,90.0,86.0,12.0,38.0,104.0,28.0,13.0,...,10.0,84.0,79.0,29.0,19.0,67.0,40.0,61.0,35.0,96.0
